# Process Data

The purpose of this `.ipynb` is to process `imu.csv` data to get `pitch`, and combine this with `gaze_positions.csv` to get `elevation`, and combine `pitch` with `elevation` to get `gaze angle`. 

## Steps
1. Establish target directory (under `data/`, select folder for session: `session_XX` where XX is a two-digit number).
2. Load `imu.csv` and `gaze_positions.csv` into dataframes.
3. Combine `pitch` with `elevation` to get `gaze angle`.
4. Save processed data to new CSV file (time step + gaze angle), as `gaze_angle.csv`.


In [45]:
import pandas as pd

# if we want to save the output file here from "Run All", make SAVE_OUTPUT = True
SAVE_OUTPUT = False

In [46]:
# 1: Establish target directory 

SESSION_ID = 'session_01'

target_dir = 'data/' + SESSION_ID

if target_dir:
    print(target_dir)
else: 
    print('Target directory doesn not exist')

data/session_01


In [47]:
# 2. Load in csv's

# Load IMU data
imu_df = pd.read_csv(f'{target_dir}/imu.csv')
# Load gaze positions data
gaze_df = pd.read_csv(f'{target_dir}/gaze_positions.csv')

print("IMU data head:")
print(imu_df.head())
print("\nGaze positions data head:")
print(gaze_df.head())

IMU data head:
        timestamp [ns]  gyro x [deg/s]  gyro y [deg/s]  gyro z [deg/s]  \
0  1760637382009784161       -4.823685       24.492264       13.326645   
1  1760637382020454161       -5.556107       27.059555       12.472153   
2  1760637382031124161       -4.886627       29.132843       11.495590   
3  1760637382041768161       -3.116608       28.949738        8.260727   
4  1760637382052413161       -2.140045       28.949738        6.185532   

   acceleration x [G]  acceleration y [G]  acceleration z [G]  roll [deg]  \
0            0.268066           -0.491699            0.718750   -5.487926   
1            0.247559           -0.487305            0.721191   -5.230553   
2            0.232910           -0.479004            0.723633   -4.919187   
3            0.186523           -0.468750            0.699707   -4.618398   
4            0.158203           -0.463379            0.697754   -4.326361   

   pitch [deg]  yaw [deg]  quaternion w  quaternion x  quaternion y  \
0   -2

In [48]:
# Downsample gaze data to IMU frame rate: for each IMU frame, find the nearest gaze elevation
imu_df_sorted = imu_df.sort_values('timestamp [ns]')
gaze_df_sorted = gaze_df.sort_values('timestamp [ns]')
merged_df = pd.merge_asof(
    imu_df_sorted,
    gaze_df_sorted[['timestamp [ns]', 'elevation [deg]']],
    on='timestamp [ns]',
    direction='nearest'
    # Optionally, set a tolerance to restrict matching distance
    # tolerance=10000  # e.g., 10,000 ns = 0.01 ms
)

# If no gaze frame is close enough, elevation will be NaN
# merged_df = merged_df.dropna(subset=['elevation [deg]'])
# merged_df = merged_df.fillna({'elevation [deg]': 0})
# Uncomment above as needed

# Now, merged_df has IMU frames with nearest gaze elevation
# Calculate gaze angle
merged_df['gaze angle [deg]'] = merged_df['pitch [deg]'] + merged_df['elevation [deg]']
gaze_angle_df = merged_df[['timestamp [ns]', 'gaze angle [deg]']].copy()
# Include pitch and elevation in final df for saving
gaze_angle_df.loc[:, 'pitch [deg]'] = merged_df['pitch [deg]']
gaze_angle_df.loc[:, 'elevation [deg]'] = merged_df['elevation [deg]']
print(f"Size of dataframe: {len(merged_df)}")
print(gaze_angle_df.head())

Size of dataframe: 6428
        timestamp [ns]  gaze angle [deg]  pitch [deg]  elevation [deg]
0  1760637382009784161        -48.705615   -29.962048       -18.743567
1  1760637382020454161        -48.772700   -30.029133       -18.743567
2  1760637382031124161        -48.824507   -30.080940       -18.743567
3  1760637382041768161        -48.870474   -30.126907       -18.743567
4  1760637382052413161        -48.900145   -30.156578       -18.743567


In [49]:
import plotly.express as px

# Calculate time in seconds, first frame is 0
gaze_angle_df = gaze_angle_df.copy()
start_ns = gaze_angle_df['timestamp [ns]'].iloc[0]
gaze_angle_df['time_sec'] = (gaze_angle_df['timestamp [ns]'] - start_ns) / 1e9

fig = px.scatter(
    gaze_angle_df,
    x='time_sec',
    y='gaze angle [deg]',
    title='Gaze Angle Over Time',
    labels={'gaze angle [deg]': 'Gaze Angle (deg)', 'time_sec': 'Time (sec)'},
    render_mode='webgl'
)
fig.update_traces(mode='lines+markers')
fig.update_yaxes(range=[-90, 45])
fig.show()

In [ ]:
from ipywidgets import widgets
from IPython.display import display

gaze_angle_df.head()
import plotly.graph_objects as go

def plot_gaze_metric(metric):
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=gaze_angle_df['time_sec'],
        y=gaze_angle_df[metric],
        mode='lines+markers',
        name=metric,
        line=dict(width=2),
        marker=dict(size=4)
    ))
    fig2.update_layout(
        title=f'{metric} Over Time',
        xaxis_title='Time (sec)',
        yaxis_title=f'{metric}',
        yaxis=dict(range=[-90, 45] if metric in ['gaze angle [deg]', 'elevation [deg]', 'pitch [deg]'] else None)
    )
    fig2.show()

dropdown = widgets.Dropdown(
    options=['elevation [deg]', 'pitch [deg]'],
    value='elevation [deg]',
    description='Metric:',
    disabled=False,
)

widgets.interact(plot_gaze_metric, metric=dropdown)

interactive(children=(Dropdown(description='Metric:', options=('elevation [deg]', 'pitch [deg]'), value='eleva…

<function __main__.plot_gaze_metric(metric)>

In [51]:
# Save entire gaze_angle_df to CSV in the target directory

if SAVE_OUTPUT == True:
    out_path = f'{target_dir}/gaze_angle.csv'
    gaze_angle_df.to_csv(out_path, index=False)
    print(f"Saved {len(gaze_angle_df)} rows to {out_path}")
else: 
    print("No output is being saved")


No output is being saved
